# MIDI processing
This notebook will handle processing the midi files into the state matrix we want for the network, and back from a network-produced state matrix to a midi.

In [1]:
import midi
import pandas as pd
import numpy as np
import pickle
import random
import os

In [2]:
def read_midis(filenames):
    '''
    A function that takes a list of filenames and returns the corresponding midi Patterns as a list.
    '''
    pattern_list = []
    for f in filenames:
        pattern = midi.read_midifile(f)
        pattern_list.append(pattern)
    return pattern_list

In [3]:
def patterns_to_matrices(pattern_list):
    '''
    Takes a list of midi Patterns and converts them to note-matrix format.
    '''
    for pattern in pattern_list:
        if pattern.tick_relative:
            pattern.make_ticks_abs() # convert to absolute times in order to sort by time.
        # constructs a dataframe 
        df = pd.DataFrame({'sixteenths':[], 'notes':[], 'velocities':[]})
        for track in pattern[1:3]:
            for event in track:
                if event.name == 'Note On':
                    df = df.append({'sixteenths': event.tick*4/pattern.resolution, 'notes': event.data[0],
                               'velocities':event.data[1]}, ignore_index=True)
        df = df.sort_values('sixteenths')
        matrix = np.zeros(((int(max(df['sixteenths'].values)) + 1), 174))
        for event in df.iterrows():
            timing = int(event[1][1])
            note = int(event[1][0]) - 21
            velocity = int(event[1][2])
            if velocity != 0:
                matrix[timing, note * 2] = 1
                matrix[timing:, note * 2 + 1] = 1
            if velocity == 0:
                matrix[timing:, note * 2 + 1] = 0
        yield matrix

In [4]:
def matrix_to_midi(matrix, save_path):
    '''
    Takes a sample in the note-matrix format and saves it as a midi to save_path.
    Returns a copy of the pattern that is written out.
    '''
    # create a pattern and add a track
    pattern = midi.Pattern()
    pattern.resolution = 480
    track = midi.Track()
    pattern.append(track)
    pattern.make_ticks_abs() 
    prev_step = np.zeros(174)
    for time, step in enumerate(matrix):
        for note, strike in enumerate(step[::2]):
            if strike == 1:
                track.append(midi.NoteOnEvent(tick = time * pattern.resolution / 4, data = [note + 21, 100]))
        for note, (sustain, last_sustain) in enumerate(zip(step[1::2], prev_step[1::2])):
            if last_sustain - sustain > 0:
                track.append(midi.NoteOnEvent(tick = time * pattern.resolution / 4, data = [note + 21, 0]))
        prev_step = step
    pattern.make_ticks_rel()
    track.append(midi.EndOfTrackEvent())
    midi.write_midifile(save_path, pattern)
    return pattern

# Processing MIDI files

Run this code to convert midi files in the folder specified by the `path` variable below to note-matrix format and save them to disk.

In [ ]:
path = 'C:/Users/Emerson/Documents/bigdata/midis/format_1/'
filepaths = [path + f for f in os.listdir(path)]

pattern_list = read_midis(filepaths)

In [ ]:
np.random.shuffle(pattern_list)
master_matrix = np.concatenate([m for m in patterns_to_matrices(pattern_list[:int(len(pattern_list)*.9)])])
test_matrix = np.concatenate([m for m in patterns_to_matrices(pattern_list[int(len(pattern_list)*.9):])])

np.save('C:/Users/Emerson/Documents/bigdata/midis/processed/mastermatrix.npy', master_matrix)
np.save('C:/Users/Emerson/Documents/bigdata/midis/processed/testmatrix.npy', test_matrix)

# Processing generated matrices

Run this code to convert the specified note-matrix sample into a .mid file.

In [42]:
sample = 'epoch 10 sample 1'
matrix = np.load('C:/Users/Emerson/Documents/bigdata/midis/generated/matrices/' + sample + '.npy')
save_path = 'C:/Users/Emerson/Documents/bigdata/midis/generated/midis/' + sample + '.mid'
pattern = matrix_to_midi(matrix, save_path)